In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Importer les données

Lien vers les différents DATASETS que propose PyTorch : https://pytorch.org/vision/stable/datasets.html

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:00<00:00, 42559790.90it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 1549275.80it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 23547595.18it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15357238.26it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Nous passons le Dataset en tant qu'argument au DataLoader. Cela encapsule un itérable sur notre ensemble de données et prend en charge le batch automatique, l'échantillonnage, le mélange aléatoire (shuffling) et le chargement de données avec plusieurs processus. Ici, nous définissons une taille de batch de 64, c'est-à-dire que chaque élément dans l'itérable du DataLoader renverra un batch de 64 caractéristiques et étiquettes.

- DataLoader : C'est un outil qui facilite la gestion de vos données en les organisant en lots (batches). Cela permet de charger les données plus efficacement et de les préparer pour l'entraînement.

- Batch automatique : Plutôt que de traiter les exemples un par un, le DataLoader regroupe plusieurs exemples dans un lot, ce qui améliore l'efficacité de l'entraînement, en particulier avec du calcul parallèle.

- Sampling (échantillonnage) : C'est la manière dont les données sont choisies dans l'ensemble de données. Par exemple, on peut échantillonner de manière aléatoire ou selon certaines règles.

- Shuffling (mélange aléatoire) : Avant de former les lots, les données peuvent être mélangées pour éviter que le modèle ne s'adapte à un ordre spécifique des données.

Caractéristiques : 

- N (Batch Size) : Le nombre d'exemples ou d'images dans un batch. C'est le nombre total d'éléments que le modèle traite simultanément pendant une itération.

- C (Channels) : Le nombre de canaux dans les données. 

- H (Height) : La hauteur de l'image (le nombre de pixels verticaux). 

- W (Width) : La largeur de l'image

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, 
                              batch_size=batch_size)
test_dataloader = DataLoader(test_data, 
                             batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Créer un model 

Pour définir un réseau de neurones dans PyTorch, nous créons une classe qui hérite de nn.Module. Nous définissons les couches du réseau dans la fonction init et spécifions comment les données passeront à travers le réseau dans la fonction forward. Pour accélérer les opérations dans le réseau de neurones, nous le déplaçons sur le GPU ou le MPS si disponible.

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):    #structure de base pour tous les modèles de réseaux de neurones dans PyTorch
    def __init__(self):
        super().__init__()    #On appelle super().__init__() pour s'assurer que toutes les fonctionnalités de nn.Module sont initialisées correctement
        self.flatten = nn.Flatten()    #couche de flattening (aplatissement) qui prend une image en entrée (par exemple, de dimensions 28x28 pixels) 
                                        #et la convertit en un vecteur 1D (de 28*28=784 éléments)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),    #couche fully connected avec 784 entrées (correspondant à l'image aplatie) et 512 sorties
            nn.ReLU(),    #Fonction d'activation ReLU
            nn.Linear(512, 512),    #Une autre couche fully connected avec 512 entrées et 512 sorties
            nn.ReLU(),
            nn.Linear(512, 10)    #La couche de sortie avec 512 entrées et 10 sorties. Ici, 10 est le nombre de classes pour la classification (par exemple, 10 classes pour des chiffres de 0 à 9 dans le dataset MNIST).
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits    #retourne les logits, c'est-à-dire les sorties non normalisées du modèle. 

#Ces logits peuvent être ensuite transformés (par exemple avec une softmax) pour obtenir des probabilités lors de la classification

# crée une instance du modèle NeuralNetwork et on déplace ce modèle vers l'appareil (CPU ou GPU) spécifié par device
model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Maintenant on souhaite optimiser les paramètres du modèle